## Fase 3: Extracción de Detalles de Actores con Selenium

El siguiente paso es extraer información detallada de los 10 principales actores de cada una de las películas extraidas en la fase 1 utilizando Selenium (desde la página e IMDB). Debereis solo tener en cuenta los 10 primeros actores de cada película o corto (en caso de que los haya)La información que deberás extraer es:

    Nombre.

    Año nacimiento.

    ¿Por qué es conocido?

    ¿Qué hace?

    Premios.

NOTA: La información de la API deberá ser almacenada en una lista de tuplas. Cada tupla corresponderá a una película. Siguiendo el siguiente ejemplo:


    [("Adam Sadler", 1966, ["El aguador", "Little Nicky", "Un papá genial", "El chico ideal" ], ["Reparto", "Producción", "Guion"] , ["American Comedy Awards, USA", "Annie Awards"],   ),  
    ...]

(
    "Nombre del actor",     # Nombre del actor (cadena de texto)
    Año de nacimiento,      # Año de nacimiento del actor (entero)
    ["Película 1", "Película 2", ...],  # Lista de películas destacadas del actor (lista de cadenas de texto)
    ["Rol 1", "Rol 2", ...],  # Lista de roles del actor (lista de cadenas de texto)
    ["Premio 1", "Premio 2", ...],  # Lista de premios importantes del actor (lista de cadenas de texto)
)

actores = [
    ("Adam Sandler", 1966, ["The Waterboy", "Little Nicky", "Big Daddy", "The Wedding Singer"], ["Actor", "Producer", "Writer"], ["MTV Movie Awards", "People's Choice Awards"]),
    ("Leonardo DiCaprio", 1974, ["Titanic", "Inception", "The Revenant", "The Wolf of Wall Street"], ["Actor", "Producer"], ["Academy Awards", "Golden Globe Awards"]),
    ("Meryl Streep", 1949, ["The Devil Wears Prada", "Sophie's Choice", "The Iron Lady", "Doubt"], ["Actress"], ["Academy Awards", "BAFTA Awards"]),
    ("Tom Hanks", 1956, ["Forrest Gump", "Cast Away", "Saving Private Ryan", "Philadelphia"], ["Actor", "Producer"], ["Academy Awards", "Golden Globe Awards"]),
    ("Jennifer Lawrence", 1990, ["The Hunger Games", "Silver Linings Playbook", "Joy", "Winter's Bone"], ["Actress"], ["Academy Awards", "Golden Globe Awards"]),
    ("Brad Pitt", 1963, ["Fight Club", "Inglourious Basterds", "The Curious Case of Benjamin Button", "Once Upon a Time in Hollywood"], ["Actor", "Producer"], ["Academy Awards", "Golden Globe Awards"]),
    ("Emma Stone", 1988, ["La La Land", "The Help", "Birdman", "Easy A"], ["Actress"], ["Academy Awards", "BAFTA Awards"]),
    ("Robert Downey Jr.", 1965, ["Iron Man", "Sherlock Holmes", "Chaplin", "Tropic Thunder"], ["Actor", "Producer"], ["Golden Globe Awards", "BAFTA Awards"]),
    ("Denzel Washington", 1954, ["Training Day", "Malcolm X", "Fences", "Flight"], ["Actor", "Director", "Producer"], ["Academy Awards", "Golden Globe Awards"]),
    ("Scarlett Johansson", 1984, ["Lost in Translation", "Marriage Story", "Avengers: Endgame", "Her"], ["Actress"], ["BAFTA Awards", "Tony Awards"])
]

In [16]:
from bs4 import BeautifulSoup
import requests

from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

from time import sleep

pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

In [36]:
lista_actores = ["Adam Sandler"]

''', "Leonardo DiCaprio", "Meryl Streep",
        "Tom Hanks","Jennifer Lawrence","Brad Pitt", "Emma Stone",
        "Robert Downey Jr.","Denzel Washington","Scarlett Johansson"]'''


# Crear un diccionario llamado "resultados_actores" para almacenar la información de cada unos de los actores
resultados_actores = {
    "nombre_actor": [],
    "anno_nacimiento": [],
    "fecha": [],
    "peliculas":[],
    "que_hace":[],
    "premios":[]
}
url_actores=(f"https://www.imdb.com/")

# Inicializa el navegador Chrome
driver = webdriver.Chrome()

# Navega a la página web "https://www.imdb.com/"
driver.get(url_actores)

driver.maximize_window()

sleep(5)

# Aceptar las cookies en la página web
driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()

sleep(5)

#driver.close()
lista_url = []

#Vamos a recorrer la lista
for actor in lista_actores:
    # Ingresar la el actor de la lista de actores y coger los datos en el campo de búsqueda y presionar Enter
    driver.find_element("css selector", "#suggestion-search").send_keys(actor, Keys.ENTER)
    
    sleep(5)
    
    #driver.execute_script("window.scrollTo(0,200)")
    
    #pinchamos en el resultado de la busqueda que nos lleva al a info del actor
    
    driver.find_element("css selector", "#__next > main > div.ipc-page-content-container.ipc-page-content-container--full.sc-dec63aac-0.uHlFE > div.ipc-page-content-container.ipc-page-content-container--center > section > div > div.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(3) > div.sc-17bafbdb-2.iUyFfD > ul > li:nth-child(1) > div.ipc-metadata-list-summary-item__c > div > a").click()
    
    #vamos a buscar porque es conocido el actor.
    
    for i in range[1,4]:
        lista_url.append(driver.find_element("xpath", f'//*[@id="js-product-list"]/div/article[{i}]/div/div[1]/h3/a').get_attribute("href"))
  


In [9]:
# Crear un diccionario llamado "resultados_actores" para almacenar la información de cada unos de los actores
resultados_actores = {
    "nombre_actor": [],
    "anno_nacimiento": [],
    "fecha": [],
    "peliculas":[],
    "que_hace":[],
    "premios":[]
}



In [ ]:
#Vamos a recorrer la lista
for actor in lista_actores:
    # Ingresar la el actor de la lista de actores y coger los datos en el campo de búsqueda y presionar Enter
    driver.find_element("css selector", "#suggestion-search").send_keys(actor, Keys.ENTER)

sleep(5)